In [1]:
import pandas as pd

In [3]:
csv_url = 'https://www.lowcarboncontracts.uk/data-portal/dataset/8a82caf8-3ec2-45a2-b1bd-581411d61cde/resource/ab7ea978-2f03-4ee4-99ef-f9cdf398b849/download/actual-cfd-generation-and-avoided-ghg-emissions.csv'

df = pd.read_csv(csv_url)

df.head()

Settlement_Date       CfD_ID Name_of_CfD_Unit Technology  \
0  2016-06-30 00:00:00  AAA-LIG-176     Charity Farm   Solar PV   
1  2016-07-01 00:00:00  AAA-LIG-176     Charity Farm   Solar PV   
2  2016-07-02 00:00:00  AAA-LIG-176     Charity Farm   Solar PV   
3  2016-07-03 00:00:00  AAA-LIG-176     Charity Farm   Solar PV   
4  2016-07-04 00:00:00  AAA-LIG-176     Charity Farm   Solar PV   

     Allocation_round Reference_Type  CFD_Generation_MWh  \
0  Allocation Round 1           IMRP              58.927   
1  Allocation Round 1           IMRP              62.795   
2  Allocation Round 1           IMRP              76.195   
3  Allocation Round 1           IMRP              82.851   
4  Allocation Round 1           IMRP              48.143   

   Avoided_GHG_tonnes_CO2e  CFD_Payments_GBP  Avoided_GHG_Cost_GBP  \
0                  18.3060           2533.46              397.0880   
1                  19.5076           2809.36              426.5806   
2                  23.6703           3871.16              517.6098   
3                  25.7381           4222.05              562.8255   
4                  14.9558           1646.81              331.7668   

   Strike_Price_GBP_Per_MWh  Market_Reference_Price_GBP_Per_MWh  \
0                     83.42                             37.5275   
1                     83.42                             36.3229   
2                     83.42                             33.3091   
3                     83.42                             34.1266   
4                     83.42                             42.5304   

   Weighted_IMRP_GBP_Per_MWh  
0                    40.4267  
1                    38.6815  
2                    32.6141  
3                    32.4596  
4                    49.2141

In [8]:
#exports
def get_financial_year(s_settlement_date):
    date_to_financial_year = lambda x: x.year if x.month > 3 else x.year-1
    
    s_settlement_date = pd.to_datetime(s_settlement_date)
    s_financial_year = s_settlement_date.map(date_to_financial_year)
    
    return s_financial_year

In [12]:
date_to_financial_year = lambda x: x.year if x.month > 3 else x.year-1

df['financial_year'] = get_financial_year(df['Settlement_Date'])

df['financial_year'].value_counts()

2020    11073
2019     8796
2021     6508
2018     4268
2017     2208
2016      376
Name: financial_year, dtype: int64

In [23]:
assert df.groupby(['CfD_ID', 'financial_year'])['Strike_Price_GBP_Per_MWh'].var().dropna().sum() == 0, 'There are differing strike price values for a single site within the same year'
df_annual_strike_prices = df.groupby(['CfD_ID', 'financial_year'])['Strike_Price_GBP_Per_MWh'].mean().reset_index().rename(columns={'CfD_ID': 'cfd_id', 'Strike_Price_GBP_Per_MWh': 'strike_price_GBP_per_MWh'})

df_annual_strike_prices

cfd_id  financial_year  strike_price_GBP_per_MWh
0    AAA-ACH-183            2019                     92.80
1    AAA-ACH-183            2020                     94.46
2    AAA-ACH-183            2021                     95.16
3    AAA-BAD-185            2018                     91.94
4    AAA-BAD-185            2019                     93.92
..           ...             ...                       ...
121  INV-WAL-001            2021                    176.57
122  INV-WAL-002            2018                    166.59
123  INV-WAL-002            2019                    170.03
124  INV-WAL-002            2020                    173.96
125  INV-WAL-002            2021                    176.57

[126 rows x 3 columns]

In [27]:
df_annual_strike_prices.to_csv('../datasets/cfd-strike-prices/cfd-strike-prices.csv', index=False, float_format='%.2f')